In [3]:
    import numpy as np
    import pandas as pd
    import os
    import matplotlib.pyplot as plt
    from keras.datasets import mnist
    from keras.layers import Input,Dense, Conv2D, MaxPooling2D, UpSampling2D,concatenate
    from keras.models import Model
    from keras.preprocessing import image
    from PIL import Image
    import tensorflow as tf
    import cv2
    from keras import layers,models
    SIZE=512


2024-09-10 14:00:24.165835: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-10 14:00:24.620898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 14:00:25.420868: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
    pip install focal-loss



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [1]:
def load_images_from_directory(directory):
    image_list = []

    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".png"):
            img = Image.open(os.path.join(directory, filename)).convert("L").resize((SIZE,SIZE))
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=-1)
            image_list.append(img_array)
    return np.array(image_list)

In [4]:
input_no_anomaly_train =load_images_from_directory("/home/oumaima/Downloads/autoencoder_local_version/train/good_train")
input_no_anomaly_test =load_images_from_directory("/home/oumaima/Downloads/autoencoder_local_version/reconstructed_images")
input_anomaly_train =load_images_from_directory("/home/oumaima/Downloads/autoencoder_local_version/train/noisy_train")
input_anomaly_test =load_images_from_directory("/home/oumaima/Downloads/autoencoder_local_version/test/noisy_test")
anomaly_train =load_images_from_directory("/home/oumaima/Downloads/noise_generator_mvtec/noise_masks") #images binaires du bruit
anomaly_test =load_images_from_directory("/home/oumaima/Downloads/autoencoder_local_version/anomaly_test") # images bianaires du bruit

In [5]:
# Définir les entrées du modèle
input_no_anomaly =Input(shape=(SIZE, SIZE, 1), name='input_no_anomaly')
input_with_anomaly = Input(shape=(SIZE, SIZE, 1), name='input_with_anomaly')

# Concaténer les deux entrées
concatenated_input = concatenate([input_no_anomaly, input_with_anomaly], axis=-1)

# Architecture du modèle U-Net
conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(concatenated_input)
conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)

up4 = UpSampling2D(size=(2, 2))(conv3)
up4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up4)
up4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up4)
merged4 = concatenate([conv2, up4], axis=3)

up5 = UpSampling2D(size=(2, 2))(merged4)
up5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)
up5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)
merged5 = concatenate([conv1, up5], axis=3)

# Sortie du modèle U-Net
anomaly_segmentation_map = Conv2D(1, (1, 1), activation='sigmoid', name='anomaly_segmentation_map')(merged5)

# Créer le modèle U-Net
unet_model = tf.keras.Model(inputs=[input_no_anomaly, input_with_anomaly], outputs=anomaly_segmentation_map, name='U-Net_Model')

# Compiler le modèle
unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Afficher le résumé du modèle
unet_model.summary()

2024-09-10 14:00:39.802371: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "U-Net_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_no_anomaly    │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_with_anomaly  │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512, 512,  │          0 │ input_no_anomaly… │
│ (Concatenate)       │ 2)                │            │ input_with_anoma… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 512,  │      1,216 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 512, 512,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 256,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 256, 256,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 256,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 128, 128,  │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 128,  │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 256, 256,  │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 256, 256,  │    295,040 │ up_sampling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 256, 256,  │    147,584 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256, 256,  │          0 │ conv2d_3[0][0],   │
│ (Concatenate)       │ 256)              │            │ conv2d_7[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 512, 512,  │          0 │ concatenate_1[0]… │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 512, 512,  │    147,520 │ up_sampling2d_1[

 Total params: 1,772,033 (6.76 MB)

 Trainable params: 1,772,033 (6.76 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
# Train the model
input_train = [input_no_anomaly_train, input_anomaly_train]
target_train = anomaly_train

input_test = [input_no_anomaly_test, input_anomaly_test]
target_test = anomaly_test

unet_model.fit(input_train, target_train, epochs=30, batch_size=12, validation_data=(input_test, target_test))


# Print the training history
print("Training complete.")
print("History:", history.history)


Epoch 1/30


2024-09-10 14:00:41.993896: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


19/19 ━━━━━━━━━━━━━━━━━━━━ 301s 16s/step - accuracy: 0.8178 - loss: 0.2989 - val_accuracy: 0.9979 - val_loss: 0.0271
Epoch 2/30
 4/19 ━━━━━━━━━━━━━━━━━━━━ 4:01 16s/step - accuracy: 0.9980 - loss: 0.0258